### Download Flowers dataset

In [0]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [0]:
#Read the dataset
import tarfile
dataset = tarfile.open('flower_photos.tgz')

In [0]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','dir','image_file'])

In [0]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, im_dir, name]

In [8]:
#Check contents of the dataframe
print('Total images: ', df.shape[0])
#df.head()

Total images:  3670


In [0]:
#extract data
dataset.extractall(path='data/')

Create Training & Test Dataset

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [0]:
train_df.to_csv('data/flower_photos/train.csv',index=False)
test_df.to_csv('data/flower_photos/test.csv', index=False)

### Load tensorflow

In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

### Read training and test data

In [0]:
import pandas as pd
train_df = pd.read_csv('data/flower_photos/train.csv')
test_df = pd.read_csv('data/flower_photos/test.csv')

In [0]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


In [0]:
#Define some parameters
img_size = 60
img_depth = 3  

### Build batch generator

In [0]:
import numpy as np

In [0]:
def batch_generator(df, batchsize=32):
    
    while True:       
                
        #Generate random numbers to pick images from dataset
        batch_nums = np.random.randint(0,df.shape[0], batchsize)
        
        #Initialize batch images array
        batch_images = np.zeros((batchsize,img_size, img_size,img_depth))
        
        #Initiate batch label array
        batch_labels = np.zeros((batchsize, len(class_names)))
        
        for i in range(batchsize):
            
            #Load image
            flower_image =  tf.keras.preprocessing.image.load_img('data/' + 
                                                                  df.loc[batch_nums[i]]['image_file'],
                                                                  target_size=(img_size, img_size))
            #Convert to array
            flower_image = tf.keras.preprocessing.image.img_to_array(flower_image)
            
            #Get the class
            img_class = df.loc[batch_nums[i]]['class']
            #Conver class to number
            img_class = class_names.index(img_class)
            #Convert class to one hot encoding
            img_class = tf.keras.utils.to_categorical(img_class, num_classes=len(class_names))
            
            #Update batch images and class arrays
            batch_images[i] = flower_image
            batch_labels[i] = img_class        
        
        yield batch_images, batch_labels         

In [0]:
a = batch_generator(train_df)
a

<generator object batch_generator at 0x7fdc9d472e60>

### Build CNN Model

In [0]:
#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add Layers
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [0]:
#Create train and test generator
batchsize = 64
train_generator = batch_generator(train_df, batchsize=batchsize) #batchsize can be changed
test_generator = batch_generator(test_df, batchsize=batchsize)

In [0]:
train_generator

<generator object batch_generator at 0x7fbbae66caf0>

In [0]:
model.fit_generator(train_generator, 
                          epochs=50,
                          steps_per_epoch= train_df.shape[0]//batchsize,
                          validation_data=test_generator,
                          validation_steps = test_df.shape[0]//batchsize)

Epoch 1/50
45/45 [==============================] - 14s 308ms/step - loss: 2.1654 - acc: 0.3507 - val_loss: 1.8489 - val_acc: 0.3267
Epoch 2/50
45/45 [==============================] - 9s 209ms/step - loss: 1.1206 - acc: 0.5441 - val_loss: 1.1856 - val_acc: 0.5284
Epoch 3/50
45/45 [==============================] - 9s 208ms/step - loss: 1.0333 - acc: 0.5872 - val_loss: 1.0588 - val_acc: 0.5895
Epoch 4/50
45/45 [==============================] - 9s 206ms/step - loss: 0.8757 - acc: 0.6635 - val_loss: 1.0498 - val_acc: 0.5540
Epoch 5/50
45/45 [==============================] - 9s 207ms/step - loss: 0.6825 - acc: 0.7521 - val_loss: 1.0562 - val_acc: 0.6207
Epoch 6/50
45/45 [==============================] - 9s 206ms/step - loss: 0.5144 - acc: 0.8212 - val_loss: 1.0469 - val_acc: 0.6009
Epoch 7/50
45/45 [==============================] - 9s 208ms/step - loss: 0.3851 - acc: 0.8712 - val_loss: 1.0880 - val_acc: 0.6293
Epoch 8/50
45/45 [==============================] - 9s 206ms/step - loss: 0